In [1]:
# import python scripts
from utils.functions_misc import *
from utils.structural import * 
from utils.functional import * 
from utils.distance import * 

In [2]:
# importing data
main_dir = "/Users/gabrielhaw/Connectome/"
# COBRE data
subject_dir = main_dir + "temp/"
participant_data = main_dir + "misc/participants_CONNECT_matched_07052025.csv"
# distance matrices
left_path = main_dir + "geodesic_distances/Left_hemisphere"
right_path = main_dir + "geodesic_distances/Right_hemisphere"

# processing distance matrices
left_dist, right_dist = average_distances(left_path, right_path, participant_data)

# loading in all final processed subject matrices
fc_mat_dir = "/Users/gabrielhaw/Connectome/subject_func_connectivity/"
sc_mat_dir = "/Users/gabrielhaw/Connectome/subject_struct_connectivity/"
sc_path_dir = "/Users/gabrielhaw/Connectome/subject_distances/"

# structural, functional connectivity & distances for each hemisphere (r: right, l: left)
lmatsc, rmatsc = struct_connect(sc_mat_dir)
lmatfc, rmatfc = funct_connect(fc_mat_dir)
llength, rlength = struct_connect(sc_path_dir)

print("all matrices loaded correctly")

all matrices loaded correctly


In [3]:
# distance-dependent consensus matrices 
print("structural:")
_, _, structural_left, _ = process_hemisphere(lmatsc, left_dist, nbins=41)
_, _, structural_right, _ = process_hemisphere(rmatsc, right_dist, nbins=41)

print("functional:")
# functional connectivity matrics
functional_left, functional_right = fconstruct_consensus(lmatfc, sthresh=0), fconstruct_consensus(rmatfc, sthresh=0)
# final correlation matrices
corrl, corrr = impute(functional_left),  impute(functional_right)

structural:
Total edges in G: 4613.0
Total edges in Gc: 2306.0
Total edges in G: 4615.0
Total edges in Gc: 2307.0
functional:
Edges removed: 0/50625 (0.0%)
Edges removed: 0/49729 (0.0%)


In [4]:
# left hemisphere embedding, Brainspace
embedding_left = gradient_(corrl, sparsity=0.9, kernel=None, n_components = 3, approach="le", z_score=True, savedf=False)
# left hemisphere embedding, Brainspace
embedding_right = gradient_(corrr, sparsity=0.9, kernel=None, n_components = 3, approach="le", z_score=True, savedf=False)

/Users/gabrielhaw/ConnectomeCode/working/.venv/lib/python3.12/site-packages/brainspace/gradient/embedding.py:206: UserWarning: Affinity is not symmetric. Making symmetric.
  warnings.warn('Affinity is not symmetric. Making symmetric.')
/Users/gabrielhaw/ConnectomeCode/working/.venv/lib/python3.12/site-packages/brainspace/gradient/embedding.py:206: UserWarning: Affinity is not symmetric. Making symmetric.
  warnings.warn('Affinity is not symmetric. Making symmetric.')


In [ ]:
# variograms
global_variogram(left_dist, structural_left, sigma=0.1, n_bins=20, binmethod="uniform")
right = variogram(right_dist, structural_right, sigma=7, n_bins=20, binmethod="uniform", plot=True)
left = variogram(left_dist, structural_left, sigma=7, n_bins=20, binmethod="uniform", plot=True)


In [ ]:
# map the gradient values to the left and right hemipsheres
right['PC1'] = right['regions'].map(embedding_right.set_index('regions')['PC1'])
left['PC1'] = left['regions'].map(embedding_left.set_index('regions')['PC1'])

# center the PC1 values, differentiate unimodal to transmodal 
left_center = left["PC1"].mean()
right_center = right["PC1"].mean()

# get the unimodal and transmodal regions
unimodal_right = right[right["PC1"] < right_center]
transmodal_right = right[right["PC1"] > right_center]

unimodal_left = left[left["PC1"] < left_center]
transmodal_left = left[left["PC1"] > left_center]

print(f"right effective range: {unimodal_right["effective_range"].mean()-transmodal_right["effective_range"].mean()}")
print(f"left effective range: {unimodal_left["effective_range"].mean()- transmodal_left["effective_range"].mean()}")
print(f"right sill: {unimodal_right["sill"].mean()-transmodal_right["sill"].mean()}")
print(f"left sill: {unimodal_left["sill"].mean()- transmodal_left["sill"].mean()}")

right effective range: -4.9869995398808555
left effective range: -6.328191154877622
right sill: 0.01897435033881789
left sill: 0.046520439198973085


In [ ]:
# find regions with high range and low sill
rank_regions_table(right, high_range=True, high_sill=False, top_n=10)

In [ ]:
# functions to determine an adequate sigma, taking at the sigma which leads to a stable rmse and r2 estimates
# df_sigma_choice_right = evaluate_sigma(right_dist, Gwr, n_bins=20, binmethod="uniform")
# df_sigma_choice_left = evaluate_sigma(left_dist, Gwl, n_bins=20, binmethod="uniform")
# create two subplots
# fig, axs = plt.subplots(2, 2, figsize=(12, 8))
# # right hemisphere
# sigma_(df_sigma_choice_right, n=1.2, axes=axs[0])
# # left hemisphere
# sigma_(df_sigma_choice_left, n=1.2, axes=axs[1])
# plt.tight_layout()
# plt.show()

In [ ]:
# to generate scatterplot
rs_plt(left, embedding1, range="effective_range", sill="sill", ax=None, save_path=None, show=True)


# normalise features across both hemispheres, for cross hemisphere comparison
combined = pd.concat([left, right])
scaler = QuantileTransformer()
scaler.fit(combined[['effective_range', 'sill']])

# preprocess and plot
gradient_left = preprocess_data(left, embedding_left, scaler)
gradient_right = preprocess_data(right, embedding_right, scaler)

dendrogram(gradient_left, save=True, savename="dendrogram_left.png")
dendrogram(gradient_right, save=True, savename="dendrogram_right.png")


# assessing trends with distance from primary regions
trend_(right, right_dist, feature="sill", hemi="right")

In [ ]:
# annotation file
annot_path = "/Users/gabrielhaw/Connectome/misc/annot_fs7/lh.lausanne500.annot"

surf_map = map_surf_values_from_annot(
    annot_path,
    region_values=left,
    label_col="regions",
    value_col="sill"
)

# center your data around the mean from the DataFrame
mean_range = left["sill"].mean()
surf_map_centered = surf_map - mean_range

# clip outliers to prevent skewing
clip_val = np.nanpercentile(np.abs(surf_map_centered), 90)
vmin = -clip_val
vmax = clip_val

plot_left_hemisphere_with_colorbar(
    surf_map=surf_map_centered,
    fsaverage=fsaverage,
    sulc_map=fsaverage_sulc,
    vmin=vmin,
    vmax=vmax,
    label='Effective range',
    output_file='left_hemisphere_plot_sill.png',
    dpi=600  # high-resolution
)